In [4]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nltk.download('vader_lexicon')
nlp = en_core_web_sm.load()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dylan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dylan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [38]:
son_dict = {'name': 'son mun san'}#, 'age': '24', 'occupation': 'student'}
lee_dict = {'name': 'lee yan ru', 'age': '24', 'occupation': 'student'}

In [18]:
new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'bankrupt': -100,
    'fraud': -100,
    'crime': -100,
    'prison': -100,
    'embezzlement': -100,
    'money_trail':-100,
    'jail': -100,
    'sentenced':-100,
    'forfeiture': -100,
    'ill-gotten gain':-100,
    'laundering':-100
}


vader = SentimentIntensityAnalyzer()

vader.lexicon.update(new_words)

In [72]:
def generate_link(identifier_dict, no_of_articles):
    link_start = "https://www.google.com/search?q="
    link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
    link_query = ""
    for identifier in identifier_dict.values():        
        link_query += identifier.replace(' ','+') + '+'
    return link_start + link_query + link_end + "&num=" + str(no_of_articles)

def generate_link_non_news(identifier_dict, no_of_articles):
    link_start = "https://www.google.com/search?q="
    link_end = '&sxsrf=AOaemvJeyEECa-gYsQeldYm25EieK_KRpQ:1631766265071&source=lnms&sa=X&ved=2ahUKEwiYuNeE04LzAhXhV3wKHclPC0IQ_AUoAHoECAEQAg&biw=1291&bih=643&dpr=2.2'
    link_query = ""
    for identifier in identifier_dict.values():        
        link_query += identifier.replace(' ','+') + '+'
    return link_start + link_query + link_end + "&num=" + str(no_of_articles)

def article_extraction(link):
    article = Article(link)
    article.download()
    try:
        article.parse()
    except:
        pass
    return article.text

def sentiment_analysis(text):
    return vader.polarity_scores(text)

def google_main_scraper(link):
    links = []
    req = Request(link, headers = {'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html5lib')
        #print(soup)
        for item in soup.find_all('div', attrs = {'class':'kCrYT'}):
            try:
                #print(item)
                raw_link = item.find('a',href = True)['href']
                raw_link = raw_link.split('/url?q=')[1].split('&sa=U&')
                links.append(raw_link)
                #print(raw_link)


            except AttributeError as e1:
                pass
            except TypeError as e2:
                pass
            except IndexError as e3:
                pass
    return links

useless_dates = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','yesterday','today']


#index is index of person
def forward_searcher(index,tags):
    for i in range(index,len(tags)):
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
    return [None,None]

def backward_searcher(index,tags):
    i = index
    while i >= 0:
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
        else:
            i -=1

def detect_age(age,lst):
    try:
        if lst[1] is not None and lst[2] is not None:
            date1 = lst[1][0]
            date2 = lst[2][0]
            if (str(age) in date1) or (str(age) in date2):
                return True
        else:

            if lst[1] == None:
                if str(age) in lst[2][0]:
                    return True

            if lst[2] == None:
                if str(age) in lst[1][0]:
                    return True
    except TypeError as e:
        pass
    
def names_and_ages(tags,age):
    result = []
    #[[name, TAG],[date, TAG]]
    check_list = []
    try:
        for i in range(len(tags)):
            #if second item is a name
            if tags[i][1] == 'PERSON':
                
                forward_age = forward_searcher(i,tags)
                backwards_age = backward_searcher(i,tags)
                new_list = [tags[i],forward_age,backwards_age]
                #new_list = [tags[i-1],tags[i],tags[i+1]]
                print(new_list)
                
                if detect_age(age,new_list) and tags[i][0] in name_list:

                    print(new_list)
                    #result += new_list
                    
                    if str(age) in new_list[1][0]:
                        check_list = [tags[i], new_list[1]]
                        print(check_list)
                    elif str(age) in new_list[2][0]:
                        print(check_list)
                        
                        check_list = [tags[i],new_list[2]]
        print(check_list)
        return confirm_age(check_list,age,3)
    except IndexError as e:
        pass 
    

    
    
                
    



1

## Test for google main page links

In [33]:
son_link = generate_link_non_news(son_dict,10)

son_scraped = google_main_scraper(son_link)

print(son_scraped)

[['https://www.treasury.gov/press-center/press-releases/Pages/jl1994.aspx', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAgQAg&usg=AOvVaw0etm02wv6ZSGfRiR-x3QFn'], ['https://pyongyangpapers.com/individual/son-mun-san/', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAcQAg&usg=AOvVaw0gm8bhdDQ3DuVj4xeKu9GL'], ['https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/954718/Democratic_People_s_Republic_of_Korea__1_.pdf', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAoQAg&usg=AOvVaw2DsJkG2YuO0ugSByJkvUly'], ['https://ofacsdnlist.com/tag/son-mun-san/', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAMQAg&usg=AOvVaw1DDplWoIettM6n78MjkVNd'], ['https://scsanctions.un.org/19181-en-dprk.html', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAkQAg&usg=AOvVaw19EzdFqvG0EzxXM1n_N7FF'], ['https://en.wikipedia.org/wiki/Kim_Jong-un', 'ved=2ahUKEwjW0oiwiIPzAhVUU30KHX1wCpQQFnoECAIQAg&usg=AOvVaw2NXa-2ehM7HRrGeMkr_fCr'], ['https://en.wikipedia.org/wiki/Joseon', 'ved=2ahUKEwjW0oiwiIPzA

In [34]:
for link in son_scraped:
    print('*******')
    print(article_extraction(link[0]))

*******
Action Targets North Korea’s Use of Deceptive Financial Practices

to Support its Weapons Programs

WASHINGTON – Today the U.S. Department of the Treasury took another step in our ongoing efforts to disrupt North Korean financial networks supporting the regime’s illicit ballistic missile and weapons of mass destruction (WMD) programs and proliferation activities. Daedong Credit Bank (DCB), together with DCB Finance Limited—a DCB front company—and DCB’s representative Kim Chol Sam were designated pursuant to Executive Order (E.O.) 13382, which targets proliferators of WMD and their supporters. The financial operations carried out by DCB, DCB Finance Limited, and Kim Chol Sam are responsible for managing millions of dollars of transactions in support of the North Korean regime’s destabilizing activities.

The Treasury Department also designated Son Mun San, the External Affairs Bureau Chief of North Korea’s General Bureau of Atomic Energy (GBAE) under E.O. 13882 for his work dire

1392–1897 Korean kingdom

Joseon (also transcribed as Chosŏn, Korean: 대조선국; 大朝鮮國, lit. 'Great Joseon State') was a Korean dynastic kingdom that lasted for approximately five centuries.[13] It was the last dynastic kingdom of Korea.[14] It was founded by Yi Seong-gye in July 1392 and replaced by the Korean Empire in October 1897.[15] The kingdom was founded following the aftermath of the overthrow of Goryeo in what is today the city of Kaesong. Early on, Korea was retitled and the capital was relocated to modern-day Seoul. The kingdom's northernmost borders were expanded to the natural boundaries at the rivers of Amnok and Tuman through the subjugation of the Jurchens.

During its 500-year duration, Joseon encouraged the entrenchment of Confucian ideals and doctrines in Korean society. Neo-Confucianism was installed as the new state's ideology. Buddhism was accordingly discouraged and occasionally faced persecutions. Joseon consolidated its effective rule over the territory of current K


*******
Get more information on the coronavirus, COVID-19 tests, and vaccine developments.


In [62]:
# Main Function
def parse(text):
    #try:
    
    doc = nlp(text)
    tags = [[X.text, X.label_] for X in doc.ents]
    labels = [x.label_ for x in doc.ents]
    items = [x.text for x in doc.ents]

    return tags

def find_names(tags):
    names = []
    for tag in tags:
        if tag[1] == 'PERSON' and tag[0] not in names:
            names.append(tag[0])
    return names
    

name_list = ['Son Mun San', 'Son', 'Mun San']



def search_articles_on_individual(individual_dict, no_of_articles):
    ###### can change to generate link news ######
    def generate_link_non_news(identifier_dict, no_of_articles):
        link_start = "https://www.google.com/search?q="
        link_end = "&sxsrf=AOaemvJeyEECa-gYsQeldYm25EieK_KRpQ:1631766265071&source=lnms&sa=X&ved=2ahUKEwiYuNeE04LzAhXhV3wKHclPC0IQ_AUoAHoECAEQAg&biw=1291&bih=643&dpr=2.2"
        link_query = ""
        for identifier in identifier_dict.values():        
            link_query += identifier.replace(' ','+') + '+'
        return link_start + link_query + link_end + "&num=" + str(no_of_articles)
    
    link = generate_link(individual_dict, no_of_articles)
    req = Request(link, headers = {'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()

    links = []
    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html5lib')
        #print(soup)
        for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
            current_dict = {}
            raw_link = (item.find('a', href = True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).get_text()
            description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            time = description.split(" · ")[0]
            #print(description)
            descript = description.split(" · ")[1]
            current_dict['title'] = title
            current_dict['time'] = time
            current_dict['description'] = descript
            current_dict['link'] = link
            current_dict['text'] = article_extraction(link)
            current_dict['sentiment'] = sentiment_analysis(article_extraction(link))
            
            parsed_description = parse(description)
            names_in_description = find_names(parsed_description)
            
            current_dict['desc_names'] = names_in_description
            
            parsed_text = parse(article_extraction(link))
            names_in_text = find_names(parsed_text)
            
            current_dict['text_names'] = names_in_text
            
            links.append(current_dict)
    return links


In [63]:
son_query = search_articles_on_individual(son_dict, 10)

In [64]:
lee_query = search_articles_on_individual(lee_dict, 10)


In [65]:
df = pd.DataFrame(lee_query)
df

,title,time,description,link,text,sentiment,desc_names,text_names
0,SMU student found guilty of molesting woman at...,2 weeks ago,"Lee Yan Ru, 25, claimed trial to molesting a w...",https://www.todayonline.com/singapore/smu-stud...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",[Lee Yan Ru],[]
1,'She was completely fine with all my advances'...,5 months ago,SINGAPORE: A 24-year-old man on trial for mole...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: A 24-year-old man on trial for mole...,"{'neg': 0.019, 'neu': 0.864, 'pos': 0.117, 'co...",[Lee Yan Ru],"[Lee Yan Ru, Lee, Josephine Chee, Ms Chee]"
2,SMU student grilled on the stand about consent...,5 months ago,Under cross-examination from Deputy Public Pro...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: A Singapore Management University (...,"{'neg': 0.029, 'neu': 0.885, 'pos': 0.087, 'co...","[Andre Chong, Lee Yan Ru]","[Andre Chong, Lee Yan Ru, Lee, Sharmila Sripat..."
3,SMU student on trial for molesting 22-year-old...,12 months ago,"Lee Yan Ru, 24, allegedly invited a young woma...",https://www.todayonline.com/singapore/smu-stud...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","[Lee Yan Ru, Netflix]",[]
4,SMU student repeatedly apologised to woman he ...,12 months ago,"The trial of student Lee Yan Ru, 24, continues...",https://www.todayonline.com/singapore/smu-stud...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","[Lee Yan Ru, Lee]",[]
5,SMU molest trial: DPP grills accused on issue ...,5 months ago,"... of Lee Yan Ru, 24, who is on trial for out...",https://www.tnp.sg/news/singapore/smu-molest-t...,Singapore Management University student Lee Ya...,"{'neg': 0.066, 'neu': 0.872, 'pos': 0.062, 'co...",[Lee Yan Ru],"[Lee Yan Ru, Andre Chong's, Lee, DPP Chong, Sh..."
6,SMU molestation trial: Accused says victim was...,5 months ago,The Singapore Management University (SMU) stud...,https://stomp.straitstimes.com/singapore-seen/...,"Jean Iau\n\nThe Straits Times\n\nApril 6, 2021...","{'neg': 0.067, 'neu': 0.884, 'pos': 0.049, 'co...",[Lee Yan Ru],"[Jean Iau\n\n, Lee Yan Ru, Lee, Josephine Chee]"
7,Man on trial for molest: I don't think she mea...,5 months ago,The Singapore Management University (SMU) stud...,https://www.tnp.sg/news/singapore/man-trial-mo...,Lee Yan Ru is accused of rubbing his exposed p...,"{'neg': 0.052, 'neu': 0.907, 'pos': 0.041, 'co...",[Lee Yan Ru],"[Lee Yan Ru, Lee, Josephine Chee]"
8,SMU molestation trial: Defence claims woman is...,12 months ago,SINGAPORE: The defence for a Singapore Managem...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: The defence for a Singapore Managem...,"{'neg': 0.076, 'neu': 0.862, 'pos': 0.062, 'co...",[Lee Yan Ru],"[Lee Yan Ru, Lee, Yan Ru, Grabbed, Tellonym]"
9,SMU molest trial: Alleged victim participated ...,12 months ago,Agree or disagree?” asked lawyer Thong Chee Ku...,https://sg.news.yahoo.com/smu-molest-trial-all...,(Yahoo News Singapore file photo)\n\nSINGAPORE...,"{'neg': 0.08, 'neu': 0.868, 'pos': 0.052, 'com...",[Lee Yan Ru],"[Yan Ru, Lee Yan Ru, Lee, Victim]"


In [67]:
for header in df['description']:
    print(parse(header))

[['Lee Yan Ru', 'PERSON'], ['25', 'DATE'], ['about 24', 'DATE']]
[['SINGAPORE', 'ORG'], ['24-year-old', 'DATE'], ['Lee Yan Ru', 'PERSON'], ['fourth-year', 'DATE'], ['SMU', 'ORG']]
[['Andre Chong', 'PERSON'], ['24-year-old', 'DATE'], ['Lee Yan Ru', 'PERSON']]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['SMU', 'ORG'], ['overnight', 'TIME']]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['a second day', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['overnight', 'TIME'], ['SMU', 'ORG'], ['2019', 'DATE']]
[['The Singapore Management University', 'ORG'], ['SMU', 'ORG'], ['Lee Yan Ru', 'PERSON'], ['24', 'DATE']]
[['The Singapore Management University', 'ORG'], ['SMU', 'ORG'], ['yesterday', 'DATE'], ['Lee Yan Ru', 'PERSON'], ['24', 'DATE']]
[['SINGAPORE', 'ORG'], ['Singapore Management University', 'ORG'], ['SMU', 'ORG'], ['24-year-old', 'DATE'], ['Lee Yan Ru', 'PERSON'], ['one', 'CARDINAL']]
[['Thong Chee Kun', 'ORG'], ['24-year-old', 'DATE'], ['Lee Yan Ru', 'PERSON']]


In [71]:
for text in df[:10]['text']:
    #print(df.iloc[[index]])
    #5th article got problem
    #insert the age here
    #print(parse(text))
    print(names_and_ages(parse(text),24))
    print("\n")

[]
None


[['Lee Yan Ru', 'PERSON'], ['fourth-year', 'DATE'], ['24-year-old', 'DATE']]
[['Lee', 'PERSON'], ['Jan 8, 2019', 'DATE'], ['22', 'DATE']]
[['Josephine Chee', 'PERSON'], ['early January 2019', 'DATE'], ['Jan 8, 2019', 'DATE']]
[['Lee', 'PERSON'], ['early January 2019', 'DATE'], ['Jan 8, 2019', 'DATE']]
[['Lee', 'PERSON'], ['that week', 'DATE'], ['early January 2019', 'DATE']]
[['Lee', 'PERSON'], ['that week', 'DATE'], ['early January 2019', 'DATE']]
[['Lee', 'PERSON'], ['that week', 'DATE'], ['early January 2019', 'DATE']]
[['Lee', 'PERSON'], ['that week', 'DATE'], ['early January 2019', 'DATE']]
[['Lee', 'PERSON'], ['Jan 8, 2019', 'DATE'], ['that week', 'DATE']]
[['Lee', 'PERSON'], ['Jan 8, 2019', 'DATE'], ['that week', 'DATE']]
[['Ms Chee', 'PERSON'], ['up to two years', 'DATE'], ['Jan 8, 2019', 'DATE']]
[['Ms Chee', 'PERSON'], ['up to two years', 'DATE'], ['Jan 8, 2019', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['Jan 8, 2019', 'DATE']]
[['Lee', 'PERSON'], [

[['Jean Iau\n\n', 'PERSON'], ['April 6, 2021', 'DATE'], None]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['April 6, 2021', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Josephine Chee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[['Lee', 'PERSON'], ['up to two years', 'DATE'], ['22', 'DATE']]
[]
None


[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], None]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], None]
[['Lee', 'PERSON'], [None, None], 

In [66]:
print(article_extraction('http://eus.tela.uk/sanctions/entity/110317'))

Information:

(Date of UN designation: 2016-11-30) (Son Mun San is the Director-General of the External Affairs Bureau of the General Bureau of Atomic Energy (GBAE), a designated entity.)
